In [ ]:
import argparse
import json
import os
import threading
import time
from random import random
import numpy as np
import requests
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
import numpy_encoder
import yaml
from sklearn.model_selection import train_test_split
import pandas as pd
import sys
import pickle

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

# setup
import pycaret
import missingno

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report, log_loss
from sklearn import preprocessing
import umap
import umap.plot
from sklearn.impute import KNNImputer
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import scipy.stats as stats

import warnings
warnings.filterwarnings('ignore')

# Loading Dataset

In [ ]:
import pathlib
current_dir = pathlib.Path.cwd()
parent_dir = current_dir.parent
result_dir = pathlib.Path.joinpath(current_dir, 'result')
pathlib.Path.mkdir(result_dir, mode=0o777, parents=True, exist_ok=True)

In [ ]:
with open('./{}'.format("data_config.json"), encoding='UTF8') as file:
    cfg = json.load(file)

drop_cols = cfg['drop_cols']
selected_cols = cfg['selected_cols']
target_col = cfg['target_col']

icu_units = list(cfg['icu_units'].values())
train_data_df = pd.DataFrame()
for icu in icu_units[:-1]:
    icu_df = pd.read_feather(parent_dir.joinpath('data', 'eicu', f'{icu}.feather'))
    train_data_df = pd.concat([train_data_df, icu_df], axis=0).reset_index(drop=True)
    
valid_data_df = pd.read_feather(parent_dir.joinpath('data', 'eicu', f'{icu_units[-1]}.feather'))

X_data, y_data = train_data_df[selected_cols], train_data_df['death']
X_valid, y_valid = valid_data_df[selected_cols], valid_data_df['death']

In [ ]:
train_data = train_data.dropna(axis=1)
len(train_data.columns)

In [ ]:
train_data_df = train_data_df[selected_cols + ['death']]
valid_data_df = valid_data_df[selected_cols + ['death']]

In [ ]:
X_data, y_data = train_data_df[selected_cols], train_data_df['death']
X_valid, y_valid = valid_data_df[selected_cols], valid_data_df['death']

In [ ]:
from fastai import *
from fastai.tabular.all import * 

In [ ]:
from pycaret.classification import *
clf1 = setup(data = train_data_df, 
             target = 'death',
             preprocess = False,
             numeric_features = selected_cols,
             silent=True)

In [ ]:
mlp = create_model('mlp')

In [ ]:
top5 = compare_models(sort='Accuracy',n_select = 5,
                      exclude = ['knn', 'svm','ridge','nb','dummy','qda']#,'xgboost']
                     )

In [ ]:
save_model(mlp,'mlp_model_11Nov2020')

In [ ]:
new_prediction = predict_model(mlp, data=valid_data_df)

In [ ]:
new_prediction

In [ ]:
evaluate_model(mlp)

In [ ]:
from pycaret.utils import check_metric
check_metric(new_prediction['death'], new_prediction['Label'], metric = 'MAE')

In [ ]:
mlp.get_params()

In [ ]:
print(mlp)